In [1]:
import boto3
from epochai.airtable.models import BenchmarkRun
from inspect_ai.log import read_eval_log
import pandas as pd

In [2]:
# Verify AWS credentials are working
session = boto3.Session()
credentials = session.get_credentials()
if credentials is None:
    print("No AWS credentials found!")
else:
    print("AWS credentials found!")

AWS credentials found!


In [3]:
# Get the runs and read the log header
runs = BenchmarkRun.all()
runs[0].logs

'https://epoch-benchmarks-production-private.s3.us-east-2.amazonaws.com/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval'

In [4]:
runs[0].model.release_date

datetime.datetime(2023, 11, 22, 0, 0)

In [5]:
runs[3].model.model_id

'gpt-4-0613'

In [6]:
len(runs)

251

In [7]:
# Convert the HTTPS URL to an S3 path
def convert_to_s3_path(url):
    # Example input: https://epoch-benchmarks-production-private.s3.us-east-2.amazonaws.com/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval
    # Example output: s3://epoch-benchmarks-production-private/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval
    from urllib.parse import urlparse
    parsed = urlparse(url)
    bucket = parsed.netloc.split('.')[0]
    path = parsed.path.lstrip('/')
    return f"s3://{bucket}/{path}"

# Try reading with the S3 path
s3_path = convert_to_s3_path(runs[0].logs)
print(f"Converting {runs[0].logs} to {s3_path}")

Converting https://epoch-benchmarks-production-private.s3.us-east-2.amazonaws.com/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval to s3://epoch-benchmarks-production-private/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval


In [8]:
run_idx = 3
log_header = read_eval_log(convert_to_s3_path(runs[run_idx].logs), header_only=True)
log_header

EvalLog(version=2, status='success', eval=EvalSpec(run_id='WsSrH8oHBm7m7vq9yTRqSC', created='2025-01-17T23:43:36+00:00', task='MATH level 5', task_id='J2ZQgRibFY3yf4C2dcaZEe', task_version=0, task_file='bench/task/hendrycks_math/__init__.py', task_attribs={}, task_args={}, solver=None, solver_args=None, tags=None, dataset=EvalDataset(name='tadamcz/hendrycks___competition_math', location='tadamcz/hendrycks___competition_math', samples=1324, sample_ids=[1, 5, 8, 9, 12, 14, 17, 21, 23, 30, 32, 36, 38, 39, 47, 50, 54, 57, 63, 71, 73, 78, 79, 84, 85, 86, 87, 93, 99, 101, 103, 105, 121, 122, 131, 133, 134, 136, 139, 142, 143, 145, 154, 157, 158, 161, 162, 163, 170, 172, 183, 187, 198, 199, 202, 205, 207, 208, 209, 213, 216, 220, 222, 227, 228, 233, 236, 240, 243, 246, 249, 250, 253, 266, 270, 272, 273, 276, 277, 280, 281, 284, 285, 292, 296, 320, 322, 326, 327, 331, 332, 339, 345, 359, 364, 371, 375, 378, 379, 381, 382, 383, 384, 387, 389, 393, 399, 404, 405, 408, 413, 415, 422, 430, 434, 43

In [9]:
log_header.stats

EvalStats(started_at='2025-01-17T23:43:36+00:00', completed_at='2025-01-18T00:53:07+00:00', model_usage={'openai/gpt-4-0613': ModelUsage(input_tokens=1982888, output_tokens=3620454, total_tokens=5603342, input_tokens_cache_write=None, input_tokens_cache_read=None), 'google/gemini-1.5-flash-002': ModelUsage(input_tokens=2843201, output_tokens=16996, total_tokens=2860197, input_tokens_cache_write=None, input_tokens_cache_read=None)})

In [10]:
log_header.eval.model

'openai/gpt-4-0613'

In [11]:
model = log_header.eval.model
log_header.stats.model_usage[model]

ModelUsage(input_tokens=1982888, output_tokens=3620454, total_tokens=5603342, input_tokens_cache_write=None, input_tokens_cache_read=None)

In [12]:
log_header.eval

EvalSpec(run_id='WsSrH8oHBm7m7vq9yTRqSC', created='2025-01-17T23:43:36+00:00', task='MATH level 5', task_id='J2ZQgRibFY3yf4C2dcaZEe', task_version=0, task_file='bench/task/hendrycks_math/__init__.py', task_attribs={}, task_args={}, solver=None, solver_args=None, tags=None, dataset=EvalDataset(name='tadamcz/hendrycks___competition_math', location='tadamcz/hendrycks___competition_math', samples=1324, sample_ids=[1, 5, 8, 9, 12, 14, 17, 21, 23, 30, 32, 36, 38, 39, 47, 50, 54, 57, 63, 71, 73, 78, 79, 84, 85, 86, 87, 93, 99, 101, 103, 105, 121, 122, 131, 133, 134, 136, 139, 142, 143, 145, 154, 157, 158, 161, 162, 163, 170, 172, 183, 187, 198, 199, 202, 205, 207, 208, 209, 213, 216, 220, 222, 227, 228, 233, 236, 240, 243, 246, 249, 250, 253, 266, 270, 272, 273, 276, 277, 280, 281, 284, 285, 292, 296, 320, 322, 326, 327, 331, 332, 339, 345, 359, 364, 371, 375, 378, 379, 381, 382, 383, 384, 387, 389, 393, 399, 404, 405, 408, 413, 415, 422, 430, 434, 436, 437, 442, 443, 444, 450, 451, 455, 463,

In [13]:
log_header.eval.task

'MATH level 5'

In [14]:
log_header.plan.config.reasoning_effort

In [15]:
log_header.eval.config.epochs

8

In [16]:
log_header.results.total_samples

10592

In [17]:
runs[run_idx].model.model_id

'gpt-4-0613'

In [18]:
runs[run_idx].scores[2].scorer

'model_graded_equiv'

In [21]:
eval_data = []
with open('data/epoch_ai_eval_log_headers.txt', 'w') as f:
    for run in runs:
        if run.logs == '':
            continue
        try:
            log_header = read_eval_log(convert_to_s3_path(run.logs), header_only=True)
        except Exception as e:
            print(f"Error reading log header for run {run.id} ({run.model.model_id}): {e}")
            continue
        f.write(str(log_header))
        f.write('\n\n')
        model_path = log_header.eval.model
        model_name = run.model.model_id
        task = log_header.eval.task
        reasoning_effort = log_header.plan.config.reasoning_effort
        epochs = log_header.eval.config.epochs
        total_samples = log_header.results.total_samples
        model_usage = log_header.stats.model_usage[model_path]
        eval_data.append({
            'Model': model_name,
            'Release Date': run.model.release_date,
            'Task': task,
            'epochs': epochs,
            'total_samples': total_samples,
            'input_tokens': model_usage.input_tokens,
            'output_tokens': model_usage.output_tokens,
            'total_tokens': model_usage.total_tokens,
        })
        print(model_path, model_usage)
df = pd.DataFrame(eval_data)
df.to_csv('data/epoch_ai_eval_data.csv', index=False)

sagemaker/01-ai/Yi-34B-Chat input_tokens=2222376 output_tokens=7308455 total_tokens=9530831 input_tokens_cache_write=None input_tokens_cache_read=None
fireworks/accounts/fireworks/models/qwen2p5-72b-instruct input_tokens=2243320 output_tokens=9708977 total_tokens=11952297 input_tokens_cache_write=None input_tokens_cache_read=None
anthropic/claude-3-5-haiku-20241022 input_tokens=468048 output_tokens=461011 total_tokens=929059 input_tokens_cache_write=0 input_tokens_cache_read=0
openai/gpt-4-0613 input_tokens=1982888 output_tokens=3620454 total_tokens=5603342 input_tokens_cache_write=None input_tokens_cache_read=None
openai/o1-mini-2024-09-12 input_tokens=988900 output_tokens=9914854 total_tokens=10903754 input_tokens_cache_write=None input_tokens_cache_read=None
openai/gpt-4-turbo-2024-04-09 input_tokens=65592 output_tokens=272569 total_tokens=338161 input_tokens_cache_write=None input_tokens_cache_read=None
sagemaker/google/gemma-2-9b-it input_tokens=1989896 output_tokens=4551564 total

In [22]:
df

,Model,Release Date,Task,epochs,total_samples,input_tokens,output_tokens,total_tokens
0,Yi-34B-Chat,2023-11-22,MATH level 5,8,10592,2222376,7308455,9530831
1,Qwen2.5-72B-Instruct,2024-09-19,MATH level 5,8,10592,2243320,9708977,11952297
2,claude-3-5-haiku-20241022,2024-10-22,GPQA Diamond,8,1584,468048,461011,929059
3,gpt-4-0613,2023-06-13,MATH level 5,8,10592,1982888,3620454,5603342
4,o1-mini-2024-09-12_high,2024-09-12,MATH level 5,4,5296,988900,9914854,10903754
...,...,...,...,...,...,...,...,...
222,gemini-1.5-flash-002,2024-09-24,OTIS Mock AIME 2024-2025,16,720,735967,634680,1370647
223,Llama-3.3-70B-Instruct,2024-12-06,OTIS Mock AIME 2024-2025,16,720,151312,806680,957992
224,DeepSeek-R1,2025-01-20,OTIS Mock AIME 2024-2025,1,45,7889,485437,493326
225,gemini-2.0-flash-001,2025-02-05,frontiermath_2025_02_28_public,16,160,10631925,732428,11364353
